In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

import tensorflow as tf
import torch

In [20]:
import warnings
warnings.simplefilter('ignore')

In [3]:
df = pd.read_csv('./data/testfiles1.csv', engine='python')[:100]
df.head()

,times,titles,contents,symbols,tags,label,wei_label
0,2020/2/11 19:54,義隆電去年每股賺8.57元 疫情不影響首季訂單,觸控晶片廠電去年營運表現亮麗，與獲利同創歷史新高，每股純益新台幣8.57元。義隆電表示，今年...,"['義隆', '中和', '2458', '1439']","['義隆', '營收']",0,1.0
1,2020/2/10 17:03,神達受新冠肺炎衝擊 2月訂單出現推遲,（3706）今（10）日公布今年1月合併營收約31億元，年增14.09%。不過，神達表示，1...,"['神達', '3706']","['神達', '疫情', '武漢肺炎']",0,-1.0
2,2020/2/10 15:16,映泰：中國子公司員工返廠困難 將機動調整訂單生產,板卡廠映泰（2399）今（10）日公告，受影響，大陸子公司因交通問題員工返廠困難，將依實際人...,"['映泰', '2399']","['疫情', '工時', '新型冠狀病毒', '武漢肺炎']",0,-1.0
3,2020/2/9 00:35,15檔迎轉機 訂單報喜,武漢肺炎疫情持續，中國大陸企業明（10）日能否順利復工、影響全球經濟走勢成為焦點，而部分產業...,"['台積電', '緯創', '旺宏', '台表科', '億光', '南六', '中化', '...","['LED', '半導體']",0,1.0
4,2020/2/6 16:59,面板重鎮疫情淪陷 訂單可望轉國內廠商,中國武漢爆發新型冠狀病毒（2019-nCoV）疫情還無法控制下來，中國許多地方政府已下令封城...,"['友達', '群創', '國產', '東和', '2409', '3481', '2504...","['面板', '光電', 'LCD']",0,1.0


In [4]:
# Get the lists of sentences and their labels.
sentences = df.contents.values
labels = df.label.values

In [53]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
vocab_directory = 'C:/Users/user11/Desktop/project/pretrained'
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(vocab_directory)

Loading BERT tokenizer...


In [26]:
new_sent = []
for sent in sentences:
    if len(sent) > 510:
        new_sent.append(sent[0:510])
    else:
        new_sent.append(sent)
#list to array
new_sent = np.array(new_sent)

In [27]:
max_len = 0

# For every sentence...
for sent in new_sent:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  502


In [28]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in new_sent:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 510,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was n

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was n

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was n

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was n

Original:  觸控晶片廠電去年營運表現亮麗，與獲利同創歷史新高，每股純益新台幣8.57元。義隆電表示，今年第1季客戶訂單並不受武漢肺炎疫情影響。
受惠客戶需求持續熱絡，義隆電去年第4季營運表現淡季不淡，季營收達27.13億元，較第3季持平表現，在業外轉投資旭暉評價利益挹注下，歸屬母公司淨利達9.82億元，季增43%，創歷史新高，每股純益3.37元。
義隆電去年總營收94.88億元，年增9.7%，在本業與業外收益同步攀升下，歸屬母公司淨利24.97億元，年增6成，每股純益8.57元。
展望未來，義隆電表示，武漢肺炎疫情蔓延，客戶第1季訂單並未因而減少，部分客戶反而還追加訂單。
義隆電指出，因應中國大陸觸控板模組外包廠春節假期延後復工，自有的台灣中和廠將支應生產，以滿足客戶需求。
法人預期，受工作天數減少影響，義隆電第1季業績將較去年第4季滑落，不過，表現應可優於去年同期水準。
Token IDs: tensor([ 101, 6240, 2971, 3253, 4275, 2449, 7442, 1343, 2399, 4245, 6880, 6134,
        4412,  778, 7927, 8024, 5645, 4363, 1164, 1398, 1201, 3644, 1380, 3173,
        7770, 8024, 3680, 5500, 5155, 4660, 3173, 1378, 2395,  129,  119, 8272,
        1039,  511, 5412, 7384, 7442, 6134, 4850, 8024,  791, 2399, 5018,  122,
        2108, 2145, 2786, 6242, 1606,  699,  679, 1358, 3636, 4031, 5511, 4142,
        4554, 2658, 2512, 7513,  511, 1358, 2669, 2145, 2786, 7444, 3724, 2898,
        5265, 4229, 5181, 8024, 5412, 7384, 7442, 1343, 2399, 5018,  125, 2108,
        4245, 6880, 6134, 4412, 

In [29]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

   90 training samples
   10 validation samples


In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
#choose batch size = 16 ,otherwise GPU will run out of memory.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [55]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
pretrained_directory = 'C:/Users/user11/Desktop/project/pretrained'
model = BertForSequenceClassification.from_pretrained(
    pretrained_directory, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

ValueError: unable to parse C:/Users/user11/Desktop/project/pretrained\config.json as a URL or as a local path

In [ ]:
import pytorch_pretrained_bert as ppb
assert 'bert-base-chinese' in ppb.modeling.PRETRAINED_MODEL_ARCHIVE_MAP

In [47]:
ppb.modeling.PRETRAINED_MODEL_ARCHIVE_MAP

{'bert-base-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz',
 'bert-large-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz',
 'bert-base-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz',
 'bert-large-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz',
 'bert-base-multilingual-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased.tar.gz',
 'bert-base-multilingual-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz',
 'bert-base-chinese': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz'}